# Week 10: Feature Selection and LASSO Regression

In this assignment, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using scikit-learn, etc.). You will:

* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second assignment, you will implement your own LASSO solver, using coordinate descent.

If you are using scikit-learn with Pandas:

The instructions may apply to other tools, but the set of parameters are specific to scikit-learn.

## 0. Load the sales dataset using Pandas:
Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 
              'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('data/kc_house_data.csv', dtype=dtype_dict)

In [4]:
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
5,7237550310,20140512T000000,1225000.0,4.0,4.50,5420.0,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760.0,101930.0
6,1321400060,20140627T000000,257500.0,3.0,2.25,1715.0,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238.0,6819.0
7,2008000270,20150115T000000,291850.0,3.0,1.50,1060.0,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650.0,9711.0
8,2414600126,20150415T000000,229500.0,3.0,1.00,1780.0,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780.0,8113.0
9,3793500160,20150312T000000,323000.0,3.0,2.50,1890.0,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390.0,7570.0


## 1. Create new features by performing following transformation on inputs:

In [5]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.


In [6]:
from sklearn import linear_model  # using scikit-learn

all_features = [
    'bedrooms',
    'bedrooms_square',
    'bathrooms',
    'sqft_living',
    'sqft_living_sqrt',
    'sqft_lot',
    'sqft_lot_sqrt',
    'floors',
    'floors_square',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built', 
    'yr_renovated'
]

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [20]:
coefs = model_all.coef_
coefs

array([    0.        ,     0.        ,     0.        ,   134.43931396,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        , 24750.00458561,     0.        ,
       61749.10309071,     0.        ,     0.        ,    -0.        ,
           0.        ])

In [22]:
import numpy as np
np.array(all_features)[np.where(coefs > 0)]

array(['sqft_living', 'view', 'grade'], dtype='<U16')

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

***ANSWER***
- Square feet living 
- View
- Grade

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [24]:
# (training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
# (training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

testing = pd.read_csv('data/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('data/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('data/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [34]:
import numpy as np
import sys

best_penalty = 10
lowest_rss = sys.float_info.max
for l1_penalty in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    results = model.predict(validation[all_features])
    rss = ((results - validation['price']) ** 2).sum()   
    if lowest_rss > rss:
        lowest_rss = rss
        best_penalty = l1_penalty
    print("Lambda: %.2f --> RSS: %.2f " % (l1_penalty, rss))
    
print("--------------\nBest lambda: %.2f --> Lowest RSS: %.2f" % (best_penalty, lowest_rss)) 

Lambda: 10.00 --> RSS: 398213327300134.88 
Lambda: 31.62 --> RSS: 399041900253346.75 
Lambda: 100.00 --> RSS: 429791604072559.50 
Lambda: 316.23 --> RSS: 463739831045121.06 
Lambda: 1000.00 --> RSS: 645898733633799.00 
Lambda: 3162.28 --> RSS: 1222506859427163.00 
Lambda: 10000.00 --> RSS: 1222506859427163.00 
Lambda: 31622.78 --> RSS: 1222506859427163.00 
Lambda: 100000.00 --> RSS: 1222506859427163.00 
Lambda: 316227.77 --> RSS: 1222506859427163.00 
Lambda: 1000000.00 --> RSS: 1222506859427163.00 
Lambda: 3162277.66 --> RSS: 1222506859427163.00 
Lambda: 10000000.00 --> RSS: 1222506859427163.00 
--------------
Best lambda: 10.00 --> Lowest RSS: 398213327300134.88


In [35]:
model = linear_model.Lasso(alpha=best_penalty, normalize=True)
model.fit(training[all_features], training['price']) # learn weights
results = model.predict(testing[all_features])
lowest_rss = ((results - testing['price']) ** 2).sum()   
print(best_penalty, lowest_rss)

10.0 98467402552698.78


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

***ANSWER***
1. 10
2. 98467402552698.78

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [0]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [0]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [0]:
l1_penalty_values = np.logspace(1, 4, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [0]:
lp = 10
rss = sys.float_info.max
lv = sys.float_info.min
sv = sys.float_info.max
for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    nz = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if nz > max_nonzeros:
        lv = l1_penalty
    if nz < max_nonzeros and sv == sys.float_info.max:
        sv = l1_penalty
    results = model.predict(validation[all_features])
    crss = ((results - validation['price']) ** 2).sum()   
    if crss < rss:
        rss = crss
        lp = l1_penalty
    print l1_penalty, rss, nz
print lp, rss    
print lv, sv

10.0 3.982133273e+14 15
14.3844988829 3.96831833944e+14 15
20.6913808111 3.96210901853e+14 15
29.7635144163 3.96210901853e+14 15
42.8133239872 3.96210901853e+14 13
61.5848211066 3.96210901853e+14 12
88.586679041 3.96210901853e+14 11
127.42749857 3.96210901853e+14 10
183.298071083 3.96210901853e+14 7
263.665089873 3.96210901853e+14 6
379.269019073 3.96210901853e+14 6
545.559478117 3.96210901853e+14 6
784.759970351 3.96210901853e+14 5
1128.83789168 3.96210901853e+14 3
1623.77673919 3.96210901853e+14 3
2335.72146909 3.96210901853e+14 2
3359.81828628 3.96210901853e+14 1
4832.93023857 3.96210901853e+14 1
6951.92796178 3.96210901853e+14 1
10000.0 3.96210901853e+14 1
20.6913808111 3.96210901853e+14
127.42749857 263.665089873


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [0]:
l1_penalty_min = 127.42749857
l1_penalty_max = 263.665089873

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [0]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [0]:
lp = 10
rss = sys.float_info.max
fmodel = None
for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    nz = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    results = model.predict(validation[all_features])
    crss = ((results - validation['price']) ** 2).sum()   
    if crss < rss and nz == max_nonzeros:
        rss = crss
        lp = l1_penalty
        fmodel = model
    print l1_penalty, rss, nz
print lp, rss  
print fmodel.coef_

127.42749857 1.79769313486e+308 10
134.597898112 1.79769313486e+308 10
141.768297655 1.79769313486e+308 8
148.938697197 1.79769313486e+308 8
156.109096739 4.40037365263e+14 7
163.279496281 4.40037365263e+14 7
170.449895824 4.40037365263e+14 7
177.620295366 4.40037365263e+14 7
184.790694908 4.40037365263e+14 7
191.96109445 4.40037365263e+14 7
199.131493993 4.40037365263e+14 7
206.301893535 4.40037365263e+14 6
213.472293077 4.40037365263e+14 6
220.642692619 4.40037365263e+14 6
227.813092162 4.40037365263e+14 6
234.983491704 4.40037365263e+14 6
242.153891246 4.40037365263e+14 6
249.324290788 4.40037365263e+14 6
256.494690331 4.40037365263e+14 6
263.665089873 4.40037365263e+14 6
156.109096739 4.40037365263e+14
[ -0.00000000e+00  -0.00000000e+00   1.06108903e+04   1.63380252e+02
   0.00000000e+00  -0.00000000e+00  -0.00000000e+00   0.00000000e+00
   0.00000000e+00   5.06451687e+05   4.19600436e+04   0.00000000e+00
   1.16253554e+05   0.00000000e+00   0.00000000e+00  -2.61223488e+03
   0.000

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [0]:
r = np.array(range(len(all_features)))
r[(fmodel.coef_ > 0) | (fmodel.coef_ < 0)]
[all_features[i] for i in r[(fmodel.coef_ > 0) | (fmodel.coef_ < 0)]]

['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade', 'yr_built']

In [0]:
all_features[1]

array([False, False,  True,  True, False, False, False, False, False,
        True,  True, False,  True, False, False, False, False], dtype=bool)